In [0]:
# Geocoding and Spatial Analysis with H3

# Setup and Configuration

In [0]:
# Required Libraries
import pandas as pd
import requests
import h3
import time
import folium
import json
import numpy as np
from typing import Dict, Tuple, List, Optional, Union
from datetime import datetime
from branca.colormap import LinearColormap
import os
from IPython.display import display
import json
import os
import hashlib
from typing import Dict, Any, Optional, Tuple

In [0]:
print("hi")

In [0]:
# Configuration Parameters
class Config:
    # API configuration
    API_KEY = ""  # Replace with your Google Maps API key
    API_RATE_LIMIT_DELAY = 0.2  # Seconds between API calls (Google standard tier = 50 QPS)
    
    # File paths
    INPUT_FILE = "geocoding/Output/Updated_RDO No. 50 - South Makati.xlsx"
    OUTPUT_DIR = "geocoding/Output"
    
    # H3 configuration
    H3_RESOLUTION = 9  # Values 0-15: higher = more precise but more hexagons
    # Resolution 9 ≈ cells with ~0.1 km² area (174m edge length)
    
    # Visualization settings
    MAP_COLORS = ['#fee5d9', '#fcae91', '#fb6a4a', '#cb181d']  # Light to dark red
    MAP_ZOOM = 14
    
    # Processing options
    SAMPLE_SIZE = None  # Set to a number (e.g., 5) to process only a sample of records
    RETRY_COUNT = 3  # Number of retries for failed geocoding
    RETRY_DELAY = 1  # Seconds to wait between retries

# Ensure output directory exists
if not os.path.exists(Config.OUTPUT_DIR):
    os.makedirs(Config.OUTPUT_DIR)


In [0]:
print(f"Configuration loaded. Using H3 resolution {Config.H3_RESOLUTION} (~{174 * (2**(9-Config.H3_RESOLUTION)):.0f}m edge length)")

# Data Loading and Preparation

In [0]:
def load_data(filepath: str, sample_size: Optional[int] = None) -> pd.DataFrame:
    """
    Load the address data from Excel file with optional sampling
    
    Args:
        filepath: Path to the Excel file
        sample_size: Number of random samples to take (None = all data)
        
    Returns:
        DataFrame containing the address data
    """
    try:
        df = pd.read_excel(filepath)
        print(f"Data loaded successfully from {filepath}")
        print(f"Total records: {len(df)}")
        
        # Clean up column names
        df.columns = df.columns.str.strip()
        
        # Take a sample if specified
        if sample_size and sample_size < len(df):
            df = df.sample(n=sample_size, random_state=42)
            print(f"Sampled {sample_size} records for processing")
            
        # Basic data validation
        required_cols = ['Street/Subdivision', 'Barangay', 'City/Municipality']
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            print(f"Warning: Missing required columns: {', '.join(missing_cols)}")
            
        # Convert ZV/SQM to numeric if it exists
        if 'ZV/SQM' in df.columns:
            df['ZV/SQM'] = pd.to_numeric(df['ZV/SQM'], errors='coerce')
            print(f"ZV/SQM values range: ₱{df['ZV/SQM'].min():,.2f} - ₱{df['ZV/SQM'].max():,.2f}")
            
        return df
    
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        # Return empty DataFrame with expected columns to prevent errors
        return pd.DataFrame(columns=['Street/Subdivision', 'Barangay', 'City/Municipality', 'Province'])

In [0]:
# Load the data
df = load_data(Config.INPUT_FILE, Config.SAMPLE_SIZE)

# Display a preview of the data
if not df.empty:
    display(df.head())
else:
    print("No data to process. Please check the input file.")

# Geocoding Functions

In [0]:
def format_address(row: pd.Series) -> str:
    """
    Format address components into a single string optimized for geocoding
    
    Args:
        row: DataFrame row containing address components
        
    Returns:
        Formatted address string
    """
    components = []
    
    # Add building/house number if available
    if pd.notna(row.get('Building/House No.')):
        components.append(str(row['Building/House No.']).strip())
    
    # Add street/subdivision
    if pd.notna(row.get('Street/Subdivision')):
        components.append(str(row['Street/Subdivision']).strip())
    
    # Add barangay (important for Philippine addresses)
    if pd.notna(row.get('Barangay')):
        components.append(f"Barangay {str(row['Barangay']).strip()}")
    
    # Add city/municipality
    if pd.notna(row.get('City/Municipality')):
        components.append(str(row['City/Municipality']).strip())
    
    # Add province
    if pd.notna(row.get('Province')):
        components.append(str(row['Province']).strip())
    
    # Always add country for better results
    components.append('Philippines')
    
    # Join with commas for standard geocoding format
    return ", ".join(components)

In [0]:
class GeocodingCache:
    """Cache for storing geocoding results to minimize API calls"""
    
    def __init__(self, cache_dir: str = "cache", cache_file: str = "geocoding_cache.json"):
        """
        Initialize the geocoding cache
        
        Args:
            cache_dir: Directory to store the cache file
            cache_file: Name of the cache file
        """
        self.cache_dir = cache_dir
        self.cache_file = os.path.join(cache_dir, cache_file)
        self.cache = {}
        self._load_cache()
    
    def _load_cache(self) -> None:
        """Load the cache from file if it exists"""
        if not os.path.exists(self.cache_dir):
            os.makedirs(self.cache_dir)
            
        if os.path.exists(self.cache_file):
            try:
                with open(self.cache_file, 'r') as f:
                    self.cache = json.load(f)
                print(f"Loaded {len(self.cache)} cached geocoding results")
            except Exception as e:
                print(f"Error loading cache: {str(e)}")
                self.cache = {}
    
    def _save_cache(self) -> None:
        """Save the current cache to file"""
        try:
            with open(self.cache_file, 'w') as f:
                json.dump(self.cache, f)
        except Exception as e:
            print(f"Error saving cache: {str(e)}")
    
    def _get_cache_key(self, address: str) -> str:
        """
        Generate a cache key for an address
        
        Args:
            address: The address to generate a key for
            
        Returns:
            A hash string to use as cache key
        """
        # Normalize the address (lowercase, remove extra spaces)
        normalized = ' '.join(address.lower().split())
        # Create a hash for the normalized address
        return hashlib.md5(normalized.encode('utf-8')).hexdigest()
    
    def get(self, address: str) -> Optional[Dict[str, Any]]:
        """
        Get cached geocoding result for an address
        
        Args:
            address: The address to look up
            
        Returns:
            Cached result or None if not in cache
        """
        key = self._get_cache_key(address)
        return self.cache.get(key)
    
    def set(self, address: str, result: Dict[str, Any]) -> None:
        """
        Store geocoding result in cache
        
        Args:
            address: The address being geocoded
            result: The geocoding result to cache
        """
        key = self._get_cache_key(address)
        self.cache[key] = result
        # Save to disk periodically (every 10 new entries)
        if len(self.cache) % 10 == 0:
            self._save_cache()
    
    def close(self) -> None:
        """Save cache and clean up resources"""
        self._save_cache()
    
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close()


In [0]:

# Update the geocode_address function to use the cache
def geocode_address(
    address: str, 
    api_key: str,
    cache: GeocodingCache,
    region: str = 'ph', 
    retry_count: int = 3, 
    retry_delay: int = 1
) -> Tuple[float, float, Dict]:
    """
    Geocode address using Google's Geocoding API with caching
    
    Args:
        address: Formatted address string
        api_key: Google Maps API key
        cache: GeocodingCache instance
        region: Region bias (default: 'ph' for Philippines)
        retry_count: Number of retries for failed requests
        retry_delay: Seconds to wait between retries
        
    Returns:
        Tuple of (latitude, longitude, full response data)
        
    Raises:
        Exception: If geocoding fails after all retries
    """
    # Check cache first
    cached_result = cache.get(address)
    if cached_result:
        print(f"Cache hit: {address}")
        location = cached_result['geometry']['location']
        return location['lat'], location['lng'], cached_result
    
    # If not in cache, proceed with API call
    print(f"Cache miss: {address}")
    if not api_key:
        raise ValueError("API key is required. Please set Config.API_KEY.")
        
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        'address': address,
        'key': api_key,
        'region': region
    }
    
    # Implement retry logic for resilience
    for attempt in range(retry_count):
        try:
            response = requests.get(url, params=params, timeout=10)
            data = response.json()
            
            if data['status'] == 'OK':
                result = data['results'][0]
                # Store in cache
                cache.set(address, result)
                
                location = result['geometry']['location']
                return location['lat'], location['lng'], result
                
            elif data['status'] == 'ZERO_RESULTS':
                raise Exception(f"No results found for address: {address}")
                
            elif data['status'] in ['OVER_QUERY_LIMIT', 'OVER_DAILY_LIMIT']:
                print(f"API quota exceeded. Status: {data['status']}")
                # Wait longer for rate limit errors
                time.sleep(retry_delay * 2)
                
            else:
                print(f"Geocoding error: {data['status']} (Attempt {attempt+1}/{retry_count})")
                time.sleep(retry_delay)
                
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {str(e)} (Attempt {attempt+1}/{retry_count})")
            time.sleep(retry_delay)
    
    # If we've exhausted all retries
    raise Exception(f"Geocoding failed after {retry_count} attempts for address: {address}")



In [0]:
def get_h3_index(lat: float, lng: float, resolution: int = 9) -> str:
    """
    Convert coordinates to H3 index at the specified resolution
    
    Args:
        lat: Latitude
        lng: Longitude
        resolution: H3 resolution (0-15, higher = more precise)
        
    Returns:
        H3 index as string
    """
    return h3.latlng_to_cell(lat, lng, resolution)

In [0]:
def h3_hexagon_to_coordinates(h3_address: str) -> List[List[float]]:
    """
    Convert H3 address to boundary coordinates for plotting
    
    Args:
        h3_address: H3 index
        
    Returns:
        List of [lat, lng] coordinates representing the hexagon boundary
    """
    # Get the boundary vertices
    boundary = h3.cell_to_boundary(h3_address)
    # Folium expects coordinates in lat,lng format
    return [[coord[0], coord[1]] for coord in boundary]

# Data Processing

In [0]:
def process_addresses(df: pd.DataFrame, api_key: str, h3_resolution: int, 
                     delay: float = 0.2) -> pd.DataFrame:
    """
    Process all addresses in the DataFrame with progress tracking and caching
    
    Args:
        df: DataFrame containing address data
        api_key: Google Maps API key
        h3_resolution: H3 resolution to use
        delay: Seconds to wait between API calls
        
    Returns:
        DataFrame with original data and geocoding results
    """
    if df.empty:
        print("No data to process.")
        return df
        
    results = []
    successful = 0
    failed = 0
    cache_hits = 0
    
    # Start time for estimating completion
    start_time = time.time()
    
    # Initialize cache
    with GeocodingCache() as cache:
        for idx, row in df.iterrows():
            try:
                # Format the address
                address = format_address(row)
                
                # Calculate and display progress
                progress = (idx + 1) / len(df) * 100
                elapsed = time.time() - start_time
                addresses_per_second = (idx + 1) / elapsed if elapsed > 0 else 0
                est_remaining = (len(df) - (idx + 1)) / addresses_per_second if addresses_per_second > 0 else 0
                
                print(f"[{progress:.1f}%] Processing {idx + 1}/{len(df)}: {address}")
                if idx > 0:
                    print(f"  Speed: {addresses_per_second:.2f} addr/sec, Est. remaining: {est_remaining/60:.1f} min")
                
                # Check if this was a cache hit before adding delay
                is_cache_hit = cache.get(address) is not None
                if is_cache_hit:
                    cache_hits += 1
                else:
                    # Only add delay for actual API calls
                    time.sleep(delay)
                
                # Geocode the address with cache
                lat, lng, google_data = geocode_address(
                    address, 
                    api_key,
                    cache,
                    retry_count=Config.RETRY_COUNT, 
                    retry_delay=Config.RETRY_DELAY
                )
                
                # Get H3 index
                h3_index = get_h3_index(lat, lng, h3_resolution)
                
                # Store results
                results.append({
                    'original_address': address,
                    'latitude': lat,
                    'longitude': lng,
                    'h3_index': h3_index,
                    'google_formatted_address': google_data['formatted_address'],
                    'google_place_id': google_data.get('place_id', ''),
                    'google_location_type': google_data['geometry']['location_type'],
                    'processing_status': 'success',
                    'from_cache': is_cache_hit
                })
                
                successful += 1
                
            except Exception as e:
                print(f"Error processing row {idx}: {str(e)}")
                
                # Store error information
                results.append({
                    'original_address': format_address(row) if 'Street/Subdivision' in row else "Address formatting failed",
                    'processing_status': 'failed',
                    'error_message': str(e)
                })
                
                failed += 1
    
    # Create results DataFrame
    results_df = pd.DataFrame(results)
    
    # Merge with original data
    if not results_df.empty:
        output_df = pd.concat([df.reset_index(drop=True), results_df.reset_index(drop=True)], axis=1)
    else:
        output_df = df.copy()
        output_df['processing_status'] = 'no_results'
    
    # Print processing summary
    print("\nProcessing Summary:")
    print(f"Total addresses processed: {len(output_df)}")
    print(f"Successful geocoding: {successful} ({successful/len(output_df)*100:.1f}%)")
    print(f"Failed geocoding: {failed} ({failed/len(output_df)*100:.1f}%)")
    print(f"Cache hits: {cache_hits} ({cache_hits/len(output_df)*100:.1f}%)")
    
    return output_df

In [0]:
# Replace the original process_addresses call with this:
if Config.API_KEY and not df.empty:
    print("Starting geocoding process with caching...")
    output_df = process_addresses(df, Config.API_KEY, Config.H3_RESOLUTION, Config.API_RATE_LIMIT_DELAY)
    
    # Save results to Excel
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_file = os.path.join(Config.OUTPUT_DIR, f"geocoded_addresses_{timestamp}.xlsx")
    output_df.to_excel(output_file, index=False)
    print(f"\nProcessing complete. Results saved to {output_file}")
    
    # Basic analysis of results
    if 'google_location_type' in output_df.columns:
        print("\nLocation type distribution:")
        print(output_df['google_location_type'].value_counts())
else:
    print("Skipping geocoding process. Either API key is missing or no data to process.")
    # For testing visualization without geocoding, we can use the original df
    output_df = df.copy()

# Visualization

In [0]:
def create_zone_value_map(df: pd.DataFrame, output_filename: str = 'zoning_map.html') -> folium.Map:
    """
    Create a map visualization of zone values using H3 hexagons
    
    Args:
        df: DataFrame with geocoded addresses and zone values
        output_filename: Name of HTML file to save the map
        
    Returns:
        Folium Map object
    """
    # Check required columns
    required_cols = ['latitude', 'longitude', 'h3_index', 'ZV/SQM']
    missing_cols = [col for col in required_cols if col not in df.columns]
    
    if missing_cols:
        print(f"Warning: Cannot create map. Missing columns: {', '.join(missing_cols)}")
        # Create an empty map centered on Philippines
        return folium.Map(location=[14.5995, 120.9842], zoom_start=11)
    
    # Clean data: Remove rows with missing required values
    map_df = df.dropna(subset=required_cols)
    
    if map_df.empty:
        print("Warning: No valid data for map visualization")
        return folium.Map(location=[14.5995, 120.9842], zoom_start=11)
    
    # Sort by ZV/SQM for better coloring
    map_df = map_df.sort_values('ZV/SQM')
    
    # Create map centered on data points
    center_lat = map_df['latitude'].mean()
    center_lng = map_df['longitude'].mean()
    m = folium.Map(location=[center_lat, center_lng], zoom_start=Config.MAP_ZOOM)
    
    # Calculate quantiles for better color distribution
    q_25 = map_df['ZV/SQM'].quantile(0.25)
    q_50 = map_df['ZV/SQM'].quantile(0.50)
    q_75 = map_df['ZV/SQM'].quantile(0.75)
    max_value = map_df['ZV/SQM'].max()
    min_value = map_df['ZV/SQM'].min()
    
    # Create color map
    colormap = LinearColormap(
        colors=Config.MAP_COLORS,
        vmin=min_value,
        vmax=max_value,
        caption='Zone Value per Square Meter (PHP)'
    )
    
    # Dictionary to store hexagons by H3 index (to avoid duplicates)
    hexagon_dict = {}
    
    # Process each row
    for idx, row in map_df.iterrows():
        try:
            # Get hexagon coordinates
            hex_coordinates = h3_hexagon_to_coordinates(row['h3_index'])
            
            # Store the highest ZV/SQM value for each hexagon
            if (row['h3_index'] not in hexagon_dict or 
                hexagon_dict[row['h3_index']]['value'] < row['ZV/SQM']):
                
                # Create popup text with all useful information
                popup_text = f"""
                <b>Address:</b> {row.get('google_formatted_address', row.get('original_address', 'N/A'))}<br>
                <b>Zone Value:</b> ₱{row['ZV/SQM']:,.2f}/sqm<br>
                """
                
                # Add classification if available
                if 'Classification' in row and pd.notna(row['Classification']):
                    popup_text += f"<b>Classification:</b> {row['Classification']}<br>"
                
                # Add barangay if available
                if 'Barangay' in row and pd.notna(row['Barangay']):
                    popup_text += f"<b>Barangay:</b> {row['Barangay']}<br>"
                
                # Store hexagon data
                hexagon_dict[row['h3_index']] = {
                    'coordinates': hex_coordinates,
                    'value': row['ZV/SQM'],
                    'popup_text': popup_text
                }
        except Exception as e:
            print(f"Error processing map row {idx}: {str(e)}")
    
    # Add hexagons to the map
    for h3_index, data in hexagon_dict.items():
        try:
            # Get color based on value
            color = colormap(data['value'])
            
            # Add polygon to map
            folium.Polygon(
                locations=data['coordinates'],
                color='black',
                weight=1,
                fill=True,
                fill_color=color,
                fill_opacity=0.7,
                popup=folium.Popup(data['popup_text'], max_width=300)
            ).add_to(m)
        except Exception as e:
            print(f"Error adding hexagon {h3_index}: {str(e)}")
    
    # Add the colormap to the map
    colormap.add_to(m)
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    # Save the map
    output_path = os.path.join(Config.OUTPUT_DIR, output_filename)
    m.save(output_path)
    print(f"Map saved to {output_path}")
    
    # Print statistics
    print("\nZone Value Statistics:")
    print(map_df['ZV/SQM'].describe())
    
    print("\nValue Ranges:")
    print(f"0-25th percentile: ₱{q_25:,.2f}/sqm")
    print(f"25-50th percentile: ₱{q_50:,.2f}/sqm")
    print(f"50-75th percentile: ₱{q_75:,.2f}/sqm")
    print(f"75-100th percentile: ₱{max_value:,.2f}/sqm")
    
    # Print classification distribution if available
    if 'Classification' in map_df.columns:
        print("\nClassification Distribution:")
        print(map_df['Classification'].value_counts())
    
    return m

In [0]:
# Create visualization if data is available
if 'ZV/SQM' in output_df.columns:
    zoning_map = create_zone_value_map(output_df, 'makati_zoning_map.html')
    display(zoning_map)
else:
    print("Cannot create visualization: ZV/SQM column missing")

# Advanced Analysis (Optional)

In [0]:
def analyze_zone_data(df: pd.DataFrame) -> None:
    """
    Perform additional analysis on zone value data
    
    Args:
        df: DataFrame with geocoded addresses and zone values
    """
    if 'ZV/SQM' not in df.columns or 'Barangay' not in df.columns:
        print("Cannot perform analysis: Required columns missing")
        return
    
    # Clean data
    analysis_df = df.dropna(subset=['ZV/SQM', 'Barangay'])
    
    if analysis_df.empty:
        print("No valid data for analysis")
        return
    
    # Analysis by Barangay
    print("\n=== Analysis by Barangay ===")
    barangay_stats = analysis_df.groupby('Barangay')['ZV/SQM'].agg([
        'count', 'min', 'max', 'mean', 'median', 'std'
    ]).sort_values('median', ascending=False)
    
    # Format currency values
    for col in ['min', 'max', 'mean', 'median']:
        barangay_stats[col] = barangay_stats[col].apply(lambda x: f"₱{x:,.2f}")
    
    print(barangay_stats)
    
    # Analysis by Classification (if available)
    if 'Classification' in analysis_df.columns:
        print("\n=== Analysis by Classification ===")
        class_stats = analysis_df.groupby('Classification')['ZV/SQM'].agg([
            'count', 'min', 'max', 'mean', 'median', 'std'
        ]).sort_values('median', ascending=False)
        
        # Format currency values
        for col in ['min', 'max', 'mean', 'median']:
            class_stats[col] = class_stats[col].apply(lambda x: f"₱{x:,.2f}")
        
        print(class_stats)

In [0]:
# Run additional analysis if data is available
if not output_df.empty and 'ZV/SQM' in output_df.columns:
    analyze_zone_data(output_df)

In [0]:
output_df

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec

# Set the style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("viridis")
plt.rcParams['figure.figsize'] = [12, 10]
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10

# Helper function to format currency values
def format_currency(x, pos):
    return f'₱{x/1000:.0f}k'

# Create barangay data
barangay_data = {
    'Barangay': [
        'POST PROPER NORTHSIDE', 'FORBES PARK VILLAGE', 'POST PROPER SOUTHSIDE', 
        'DASMARIÑAS VILLAGE', 'URDANETA VILLAGE', 'PINAGKAISAHAN', 
        'GUADALUPE NUEVO', 'BEL-AIR (SALCEDO VILLAGE)', 'WEST REMBO',
        'SOUTH CEMBO', 'EAST REMBO', 'PITOGO', 'CEMBO', 'COMEMBO', 'PEMBO', 'RIZAL'
    ],
    'count': [270, 38, 783, 43, 75, 52, 65, 500, 93, 87, 108, 44, 34, 45, 131, 133],
    'min': [200000, 154000, 98000, 275000, 0, 0, 0, 0, 83000, 103000, 75000, 101000, 70000, 65000, 59000, 47000],
    'max': [900000, 414000, 900000, 425000, 929000, 300000, 300000, 940000, 150000, 250000, 180000, 214000, 200000, 138000, 123000, 79000],
    'median': [500000, 338000, 300000, 300000, 222000, 123000, 120000, 115000, 115000, 113000, 110000, 108000, 80000, 65000, 60000, 47000]
}

# Create classification data
classification_data = {
    'Classification': ['CR', 'CC', 'RC', 'RR', 'X', 'PS'],
    'count': [839, 246, 205, 878, 77, 256],
    'min': [0, 0, 0, 0, 0, 0],
    'max': [940000, 423000, 550000, 350000, 575000, 385000],
    'median': [500000, 180000, 160000, 120000, 113000, 98000],
    'full_name': ['Commercial Regular', 'Commercial Center', 'Residential Commercial', 
                 'Residential Regular', 'Special Zone', 'Public Service']
}

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter

# Set the style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("viridis")

# Helper function to format currency values
def format_currency(x, pos):
    return f'₱{x/1000:.0f}k'

# Create barangay data
barangay_data = {
    'Barangay': [
        'POST PROPER NORTHSIDE', 'FORBES PARK VILLAGE', 'POST PROPER SOUTHSIDE', 
        'DASMARIÑAS VILLAGE', 'URDANETA VILLAGE', 'PINAGKAISAHAN', 
        'GUADALUPE NUEVO', 'BEL-AIR (SALCEDO VILLAGE)', 'WEST REMBO',
        'SOUTH CEMBO', 'EAST REMBO', 'PITOGO', 'CEMBO', 'COMEMBO', 'PEMBO', 'RIZAL'
    ],
    'count': [270, 38, 783, 43, 75, 52, 65, 500, 93, 87, 108, 44, 34, 45, 131, 133],
    'min': [200000, 154000, 98000, 275000, 0, 0, 0, 0, 83000, 103000, 75000, 101000, 70000, 65000, 59000, 47000],
    'max': [900000, 414000, 900000, 425000, 929000, 300000, 300000, 940000, 150000, 250000, 180000, 214000, 200000, 138000, 123000, 79000],
    'median': [500000, 338000, 300000, 300000, 222000, 123000, 120000, 115000, 115000, 113000, 110000, 108000, 80000, 65000, 60000, 47000]
}

# Create classification data
classification_data = {
    'Classification': ['CR', 'CC', 'RC', 'RR', 'X', 'PS'],
    'count': [839, 246, 205, 878, 77, 256],
    'min': [0, 0, 0, 0, 0, 0],
    'max': [940000, 423000, 550000, 350000, 575000, 385000],
    'median': [500000, 180000, 160000, 120000, 113000, 98000],
    'full_name': ['Commercial Regular', 'Commercial Center', 'Residential Commercial', 
                 'Residential Regular', 'Special Zone', 'Public Service']
}

# Convert to DataFrames
df_barangay = pd.DataFrame(barangay_data)
df_classification = pd.DataFrame(classification_data)

# Sort barangay data by median value (descending)
df_barangay = df_barangay.sort_values('median', ascending=False)

In [0]:

# --------------------------------------------------------------------------
# PLOT 1: Bar Chart for Median Values by Barangay
# --------------------------------------------------------------------------
plt.figure(figsize=(12, 10))
bars = sns.barplot(x='median', y='Barangay', data=df_barangay, palette='Blues_d')
plt.title('Median Zone Values by Barangay', fontweight='bold', fontsize=16)
plt.xlabel('Median Zone Value (₱/sqm)', fontsize=12)
plt.ylabel('Barangay', fontsize=12)
plt.gca().xaxis.set_major_formatter(FuncFormatter(format_currency))

# Add value annotations
for i, bar in enumerate(bars.patches):
    value = df_barangay.iloc[i]['median']
    plt.text(bar.get_width() + 20000, bar.get_y() + bar.get_height()/2, 
             f'₱{value:,.0f}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

In [0]:

# --------------------------------------------------------------------------
# PLOT 2: Bar Chart for Median Values by Classification
# --------------------------------------------------------------------------
plt.figure(figsize=(10, 8))
class_colors = plt.cm.viridis(np.linspace(0, 0.9, len(df_classification)))
bars2 = sns.barplot(x='Classification', y='median', data=df_classification, palette=class_colors)
plt.title('Median Zone Values by Classification', fontweight='bold', fontsize=16)
plt.xlabel('Classification', fontsize=12)
plt.ylabel('Median Zone Value (₱/sqm)', fontsize=12)
plt.gca().yaxis.set_major_formatter(FuncFormatter(format_currency))

# Add value annotations
for i, bar in enumerate(bars2.patches):
    value = df_classification.iloc[i]['median']
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 20000, 
             f'₱{value:,.0f}', ha='center', fontsize=10)

# Add classification full names as a legend
handles = [plt.Rectangle((0,0),1,1, color=class_colors[i]) for i in range(len(df_classification))]
labels = [f"{row['Classification']} - {row['full_name']}" for _, row in df_classification.iterrows()]
plt.legend(handles, labels, loc='upper right')

plt.tight_layout()
plt.show()

In [0]:
# --------------------------------------------------------------------------
# PLOT 3: Pie Chart for Property Distribution by Classification
# --------------------------------------------------------------------------
plt.figure(figsize=(10, 8))
explode = [0.1] + [0] * (len(df_classification) - 1)  # Explode first slice (CR)
wedges, texts, autotexts = plt.pie(
    df_classification['count'], 
    autopct='%1.1f%%', 
    explode=explode,
    colors=class_colors,
    shadow=True, 
    startangle=90
)
plt.title('Property Distribution by Classification', fontweight='bold', fontsize=16)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle

# Create custom legend
legend_labels = [f"{row['Classification']} - {row['full_name']} ({row['count']} properties)" 
                for _, row in df_classification.iterrows()]
plt.legend(wedges, legend_labels, loc='center left', bbox_to_anchor=(1, 0.5))

plt.tight_layout()
plt.show()

In [0]:
# --------------------------------------------------------------------------
# PLOT 4: Key Insights Textbox
# --------------------------------------------------------------------------
plt.figure(figsize=(12, 6))
plt.axis('off')
insights_text = """
KEY INSIGHTS:
• Premium Areas: POST PROPER NORTHSIDE, FORBES PARK VILLAGE, and DASMARIÑAS VILLAGE have the highest median values (₱300,000+/sqm).
• Commercial Impact: Commercial Regular (CR) properties command the highest values (₱500,000/sqm median), 2.8× higher than Residential Regular (RR).
• Property Distribution: Residential Regular (RR) and Commercial Regular (CR) properties make up the majority (68%) of all properties.
• Value Range: There's a 10× difference in median values between the highest (POST PROPER NORTHSIDE: ₱500,000) and lowest (RIZAL: ₱47,000) barangays.
"""
plt.text(0.5, 0.5, insights_text, ha='center', va='center', fontsize=14, 
         bbox=dict(facecolor='lightblue', alpha=0.1, boxstyle='round,pad=1'))

plt.title('MAKATI CITY ZONE VALUE ANALYSIS - KEY INSIGHTS', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [0]:
# --------------------------------------------------------------------------
# PLOT 5: Range Comparison (Bonus Visualization)
# --------------------------------------------------------------------------
plt.figure(figsize=(12, 10))
y_pos = np.arange(len(df_barangay))

# Create custom colors that are lighter at the top and darker at bottom
colors = plt.cm.Blues(np.linspace(0.3, 0.9, len(df_barangay)))

# Plot median values as bars
bars = plt.barh(y_pos, df_barangay['median'], color=colors)

# Add error bars from min to max
plt.errorbar(df_barangay['median'], y_pos, 
             xerr=[[df_barangay['median'] - df_barangay['min']], [df_barangay['max'] - df_barangay['median']]], 
             fmt='o', color='black', capsize=5)

# Customize the plot
plt.yticks(y_pos, df_barangay['Barangay'])
plt.xlabel('Zone Value (₱/sqm)')
plt.title('Zone Value Ranges by Barangay', fontsize=16, fontweight='bold')
plt.gca().xaxis.set_major_formatter(FuncFormatter(format_currency))

# Add range annotations
for i, bar in enumerate(bars):
    min_val = df_barangay.iloc[i]['min']
    max_val = df_barangay.iloc[i]['max']
    plt.text(bar.get_width() + 10000, bar.get_y() + bar.get_height()/2, 
             f'₱{min_val:,.0f} - ₱{max_val:,.0f}', va='center', fontsize=9)

plt.tight_layout()
plt.show()